## Code fourni pour lire les binaires:

In [144]:
import matplotlib.pyplot as plt
import numpy as np
import time

def read_int(f):
    ba = bytearray(4)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.int32)
    return prm[0]

def read_double(f):
    ba = bytearray(8)
    f.readinto(ba)
    prm = np.frombuffer(ba, dtype=np.double)
    return prm[0]

def read_double_tab(f, n):
    ba = bytearray(8*n)
    nr = f.readinto(ba)
    if nr != len(ba):
        return []
    else:
        prm = np.frombuffer(ba, dtype=np.double)
        return prm

def get_pics_from_file(filename):
    # Lecture du fichier d'infos + pics detectes (post-processing KeyFinder)
    #print("Ouverture du fichier de pics "+filename)
    f_pic = open(filename, "rb")
    info = dict()
    info["nb_pics"] = read_int(f_pic)
    #print("Nb pics par trame: " + str(info["nb_pics"]))
    info["freq_sampling_khz"] = read_double(f_pic)
    #print("Frequence d'echantillonnage: " + str(info["freq_sampling_khz"]) + " kHz")
    info["freq_trame_hz"] = read_double(f_pic)
    #print("Frequence trame: " + str(info["freq_trame_hz"]) + " Hz")
    info["freq_pic_khz"] = read_double(f_pic)
    #print("Frequence pic: " + str(info["freq_pic_khz"]) + " kHz")
    info["norm_fact"] = read_double(f_pic)
    #print("Facteur de normalisation: " + str(info["norm_fact"]))
    tab_pics = []
    pics = read_double_tab(f_pic, info["nb_pics"])
    nb_trames = 1
    while len(pics) > 0:
        nb_trames = nb_trames+1
        tab_pics.append(pics)
        pics = read_double_tab(f_pic, info["nb_pics"])
    #print("Nb trames: " + str(nb_trames))
    f_pic.close()
    return tab_pics, info

    """
    ######### Pics ############
    # NO KEY
    plt.figure(1)
    plt.subplot(211)
    plt.plot(range(1,info["nb_pics"]+1), np.mean(pics_nokey,axis=0), 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('no key')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    # PAD-0
    plt.subplot(212)
    plt.plot(range(1,info["nb_pics"]+1), pics_nokey[0], 'ko')
    plt.xlabel('numéro de pic')
    plt.ylabel('valeur du pic')
    plt.title('PAD-0')
    plt.ylim(0, 1.5)
    plt.grid(b=True, which='both')
    #
    plt.show()
    """

# Notre Algorithme
* Séparer les trames en chunk ------- While == précédent + [un certain pas]
* Attention à shift+a par exemple --- Maybe in top best letters ?
* ctrl alt suppr au début ---- Histoire de pas oublier

### Bin to png

In [145]:
#for file in glob.glob("../data/pics_*.bin"):
 #   key = file.split("pics_")[1].replace(".bin","")
 #   values, info = get_pics_from_file(file)
 #   plt.imshow(values,aspect="auto",)
 #   plt.savefig("../data/PNGs/pics_" + key + ".png")

### Bibliothèques utilisées :

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
import glob

## Constitution du dataset

In [159]:
# Il faut enlever le bruit

X,Y = [],[]

pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")

letters = 0
nokey = np.mean(pics_nokey,axis=0)
for file in glob.glob("../data/pics_*.bin"):
    key = file.split("pics_")[1].replace(".bin","")
    if key == "LOGINMDP":
        continue
    values, info = get_pics_from_file(file)
    np.random.shuffle(values)
    values = values[:3000]
    values = np.subtract(values[:3000],nokey)
    mean = np.mean(values,axis=0)
    #values = values * mean
    values = np.subtract(values,nokey)
    for frame in values:
        X.append(frame)
        Y.append(key)
    letters += 1
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

# Scaling des données
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

### KNN Algorithm

In [160]:
import math
# Best value for N (internet said this)
N = math.floor(math.sqrt(len(Y_test)))
if N % 2 == 0:
    N -= 1
N

157

In [ ]:
# KNN Clustering
classifier = KNeighborsClassifier(n_neighbors=40, p=2, metric='euclidean',n_jobs=-1)
classifier.fit(X_train,Y_train)
Y_pred = classifier.predict(X_test)
Y_pred

### Evaluating model ###
print(Y_test)
print(accuracy_score(Y_test,Y_pred),"%")
cm = confusion_matrix(Y_test,Y_pred)
# La diagonale c'est les bons guess et le reste les fautes
#print(cm)

In [ ]:
""" Code poubelle -> tests
pics_nokey, info = get_pics_from_file("../data/pics_NOKEY.bin")
values, info = get_pics_from_file("../data/pics_3.bin")
print(np.mean(values,axis=0))
np.random.shuffle(values)
plt.imshow(values[:3000],aspect="auto")
plt.show()
nokey = np.mean(pics_nokey,axis=0)
denoised = np.subtract(values[:3000],nokey)
plt.imshow(denoised,aspect="auto")
plt.show()
mean = np.mean(values[:3000],axis=0)
cleaned = values[:3000] * mean
plt.imshow(cleaned,aspect="auto")
plt.show()

denoised = np.subtract(cleaned,nokey)
plt.imshow(denoised,aspect="auto")
plt.show()
"""

In [ ]:
loginmdp, info = get_pics_from_file("../data/pics_LOGINMDP.bin")